# Lecture 1030

More pandas and charts with Altair using Berkeley 311 call data.

## Import modules
As usual, we'll import modules at the top of the notebook. This time, we don't need the `requests` module since we're not going to re-download the data from the Internet.

### What is Altair?

[Altair](https://altair-viz.github.io/) is a data visualization library for Python. `matplotlib` is usually the first data viz module Python programmers learn, but Altair is easier to use. The Altair community uses the alias `alt` when importing.

In [ ]:
import pandas as pd
import altair as alt

## Import data

We did a lot of work last week cleaning up the Berkeley 311 calls. We don't need to redo that work since we exported a clean version called `berkeley_311_clean.csv`. 

Remember that a `csv` file is just a plain-text file. That means that the file, just as it is, cannot retain the **dtype** of a column.

So this time when we import the data, we'll want to make sure that we set up the dtypes we do know and parse `datetime` dtypes correctly.

I also want to set **Case_ID** to an `object` dtype instead of an `int` dtype. Why would I want to do this? You can't operate on **Case_ID** like it's a number. You aren't going to add up the Case_IDs. So it's better to import that column as an `object`.

Last thing: where you saved this notebook file matters. Where does the file `berkeley_311_clean.csv` exist locally on your computer?

In [ ]:
berkeley_311 = pd.read_csv('../1023/berkeley_311_clean.csv', 
    dtype={
        'Case_ID': object,
    },
    parse_dates=['Date_Opened', 'Date_Closed', 'Close_Time']
)

In [ ]:
berkeley_311.head()

In [ ]:
berkeley_311.info()

The **Close_Time** column didn't get typed as `timedelta`. It doesn't look like it's possible to do so with `pd.read_csv()`. (There's [an open issue](https://github.com/pandas-dev/pandas/issues/8185) on the pandas repo as of today's lecture.) So we'll just set it this way:

In [ ]:
berkeley_311['Close_Time'] = pd.to_timedelta(berkeley_311['Close_Time']) 

In [ ]:
berkeley_311.info()

In [ ]:
berkeley_311

## Explore data

What do I do if I don't have a question yet? I'm not really sure what to look into with this 311 data. So I'm going to explore it a little bit. I might do some analysis, I might not.

### Categories of incidents in 2022

I'm curious about the different categories of incidents in the year 2022.

First, I'll create a new dataframe `berkeley_311_2022` that subsets the `berkeley_311` data to just the cases that were open in 2022. (We discussed this last week, but subsetting data is a way to filter data.)

In [ ]:
berkeley_311_2022 = berkeley_311.loc[
    (berkeley_311['Date_Opened'] >= '2022-01-01') &
    (berkeley_311['Date_Opened'] < '2023-01-01') # Why don't I use `berkeley_311['Date_Opened'] <= '2022-12-31']` ?
].copy()

Let's look at that expression above. 
- I used [`df.loc[ expression ]` because it is more performant than subsetting using just `df[ expression ]`. Either way is fine for your work. There are many ways to subset data in pandas; here's some more information about [those ways](https://pandas.pydata.org/docs/user_guide/indexing.html).
- We're using `&` (instead of `and`). Remember our first lectures: `&` is a [bitwise operator](https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations), while `and` is a [logical or boolean operator](https://docs.python.org/3/reference/expressions.html#boolean-operations).

In [ ]:
berkeley_311_2022

One thing I'm seeing immediately is that the index of this new dataframe `berkeley_311_2022` looks kind of weird. It's no longer sequential. I can reset the index to make it sequential by using `df.reset_index(drop=True)`.

```python
berkeley_311_2022 = berkeley_311_2022.reset_index(drop=True)
```

Alternatively, instead of copying the original dataframe with df.copy(), we can reset the index at the same time we subset the data:

In [ ]:
berkeley_311_2022 = berkeley_311.loc[
    (berkeley_311['Date_Opened'] >= '2022-01-01') &
    (berkeley_311['Date_Opened'] < '2023-01-01') 
].reset_index(drop=True)

In [ ]:
berkeley_311_2022

#### Let's view all the unique values of **Request_Category**

You can call `series.unique()` on a column:

In [ ]:
berkeley_311_2022['Request_Category'].unique()

I'm interested in getting a count of those categories for 2022. How can I achieve this? We'll use the method `series.value_counts()`.

In [ ]:
berkeley_311_2022['Request_Category'].value_counts()

You can also get the `value_counts()`  for two columns:

In [ ]:
berkeley_311_2022[['Request_Category', 'Request_SubCategory']].value_counts()

In [ ]:
berkeley_311_2022[['Request_Category', 'Request_SubCategory']].value_counts()

OK, so let's just look at the major topline categories.

In [ ]:
category_counts_2022 = berkeley_311_2022['Request_Category'].value_counts()
category_counts_2022

#### Convert series to dataframe
The `.value_counts()` method creates a series, not a dataframe. We'll convert that to a pandas dataframe with `to.frame()`.

In [ ]:
category_counts_2022 = category_counts_2022.to_frame()
category_counts_2022

#### Resetting the index

In this dataframe, the index is no longer a series of sequential integers like we've seen before. We'll convert **Request_Category** to a column, from an index. That will make the dataframe easier to use later.

We're going to use `df.reset_index()`. This time, we're not going to use the `drop=True` argument because we want to create a wholly new index.

In [ ]:
category_counts_2022 = category_counts_2022.reset_index()

In [ ]:
category_counts_2022

Looks like `Refuse and Recycling`, along with `General Questions/information` and `Streets, Utilities, and Transportation` were among the top issues in 2022. Might be worth looking into some of the sub-categories later.

#### Rename columns

Let's change the column names, while we're at it.

You can replace _all_ the columns in a dataframe in one sweep with the following code:

```python
category_counts_2022.columns  = ['category', 'cases']
```

If you have a lot of columns, that's going to be a long list. But if you have a lot of columns to rename, the method above might be easier. If you have only one column to rename out of many columns, you'll want to use the following code:

```python
category_counts_2022.rename(columns={'Request_Category': 'category'}, inplace=True)
```

The first argument in the `df.rename()` method is `columns`. And what do we set columns to? We set it to a Python dictionary where the "key" is the original column name and the "value" is the new column name: `{'Request_Category': 'category'}`. 

The second argument is `inplace=True`. That tells us to change the `category_counts_2022` "in place" or without having to reset the dataframe variable. A lot of the methods in pandas return a new dataframe instead of altering the original dataframe. An alternative to using `inplace` is the following code:

```python
category_counts_2022 = category_counts_2022.rename(columns={'Case_ID': 'Count'})
```

In [ ]:
category_counts_2022.columns  = ['category', 'cases']
category_counts_2022

#### Let's visualize this summary table!

Before we run the Altair code below, let's take a closer look:

```python
alt.Chart(category_counts_2022).mark_bar().encode(
    x='cases',
    y='category'
).properties(
    title='Berkeley 311 cases in 2022'
)
```
The first part of the code `alt.Chart()` requires you to fill the first argument with a dataframe, in this case `category_counts_2022`.

The next part of the code `mark_bar()` specifies a bar chart. (If you want a line chart, you'd use `mark_line()`.)

After that, `.encode()` tells Altair which columns to use for the `x` and `y` axes.

If you want to add a title, you'd use Altair's `.properties()` method.

In [ ]:
alt.Chart(category_counts_2022).mark_bar().encode(
    x='cases',
    y='category'
).properties(
    title='Berkeley 311 cases in 2022'
)

Annoyingly, this doesn't sort the chart in descending order, which I prefer. This is the code to do, it's a little more complicated:

```python
alt.Chart(category_counts_2022).mark_bar().encode(
    x='cases',
    y=alt.Y('category', sort='-x')
).properties(
    title='Berkeley 311 cases in 2022'
)
```

Basically, you have to create a custom Y encoding with the format: `alt.Y('column_name', sort='-x')`. `-x` means the inverse of the x-axis, in this case. This is not intuitive, I think — it's just something you'd have to look up in the documentation.

In [ ]:
alt.Chart(category_counts_2022).mark_bar().encode(
    x='cases',
    y=alt.Y('category', sort='-x')
).properties(
    title='Berkeley 311 cases in 2022'
)

### Count how many incidents per year

The next thing I'd like to do is get a count of all the incidents by year. However, I know from the last notebook that the data for 2010 and 2023 are not complete. So I need to subset.

Below, I'm creating a new dataframe called `berkeley_311_complete` that limits the `berkeley_311` dataframe to ones in which the **Date_Opened** value starts on or after January 1, 2011 and is before January 1, 2023. 

In [ ]:
berkeley_311_complete = berkeley_311.loc[
    (berkeley_311['Date_Opened'] >= '2011-01-01') &
    (berkeley_311['Date_Opened'] < '2023-01-01')
].reset_index(drop=True)

#### Aggregate with `df.groupby()`

To aggregate the data, we're going to use a method called `df.groupby()`. Normally, when we group data, we'll group them by columns, like so:

```python
df.groupby(['Column 1', 'Column 2'])
```

You can also just group by a single column, like we're doing below:

In [ ]:
berkeley_311_complete.groupby(['Request_Category'])

Running a `df.groupby()` doesn't do anything on its own, it just creates a pandas DataFrameGroupBy object. You have to follow it up with some kind of other method. Below, we're calling `df.count()` on the DataFrameGroupBy object.

In [ ]:
berkeley_311_complete.groupby(['Request_Category']).count()

It's kind of like getting `value_counts()` on a column.

OK! So that's a new dataframe, with a little too much info. We're not going to do anything with this particular dataframe; I just wanted to show you how `groupby()` works so we can look specifically at how to use it for datetimes.

#### Use df.groupby() with datetimes

Now that we know a little bit about the `groupby()` method, let's figure out how to use this with dates.

It's a little tricky to group by datetimes. Instead of grouping by just a column name, we're going to have to use a method called `pd.Grouper`. 

Before we run the code below, let's look at the different arguments within the method:

```python
pd.Grouper(key='Date_Opened', axis=0, freq='A')
```

The `key` argument lists the column. The `axis` argument is `0`. In pandas, axis 0 is rows and axis 1 means columns. That means you can do column-wise calculations if your data is shaped differently. 

The `freq` argument is `A`, which stands for "annual" or year (`Y` also works, but isn't documented). You can see other [frequency arguments](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) in the official pandas documentation.

In [ ]:
berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A')])

Remember that running a `df.groupby()` doesn't do anything on its own; you have to chain that command with some kind of other method. Below, we're calling `df.count()` on the DataFrameGroupBy object. Finally, we're calling our new dataframe `annual_cases`.

In [ ]:
annual_cases = berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A')]).count()

In [ ]:
annual_cases

Now let's subset just the one column, **Case_ID**, from annual cases, then reset the index so that `Date_Opened` becomes a new column:

In [ ]:
annual_cases = annual_cases[['Case_ID']].reset_index()
annual_cases

#### Rename columns

In [ ]:
annual_cases.rename(columns={'Case_ID': 'cases'}, inplace=True)

Let's take a look at our nicely named summary table:

In [ ]:
annual_cases

Let's create a new column in `annual_cases` called **Year**.

In [ ]:
annual_cases['year'] = annual_cases['Date_Opened'].dt.year

In [ ]:
annual_cases

At this point, I don't need the **Date_Opened** column anymore. So I can subset the dataframe with just the two columns I need. 

In [ ]:
annual_cases = annual_cases[['year', 'cases']].copy()

In [ ]:
annual_cases

#### Visualize

In [ ]:
alt.Chart(annual_cases).mark_bar().encode(
    x='year',
    y='cases'
)

That's pretty cool, but **Year** shows up kind of weird. Let's make a very small alteration to the code.

Before you run the code below, notice that after `Year` there's a colon and an `O`. The `O` is shorthand for "ordinal," and tells Altair to treat `Year` as if it's a discrete quantity (a.k.a. integers), not a continuous quantity (e.g. a number with decimals). 

In [ ]:
alt.Chart(annual_cases).mark_bar().encode(
    x='year:O',
    y='cases'
).properties(
    title='Berkeley 311 calls: Number of cases'
)

You can read about more [Altair encoding types](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types) in the documentation. It's helpful to get familiar with those encoding types in the event your chart doesn't look quite right. Try adjusting the encoding types on your own to see what happens.

### Median Close_Time by year

Now I'd like to try to get the median length of time it takes to close a case by year. I'm going to try something I think will work...

In [ ]:
median_close_time = berkeley_311_complete.groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A') ]).median()

It looks like that didn't work! Sometimes pandas doesn't work the way you want it to. The problem is that we have too many columns that don't support calculating a median (for example, a bunch of text-only columns.) So we'll have to subset the dataframe for just the two columns we want. Then we can run the `groupby()` operation.

In [ ]:
median_close_time = berkeley_311_complete[
    ['Date_Opened', 'Close_Time']
].groupby([pd.Grouper(key='Date_Opened', axis=0, freq='A') ]).median()

median_close_time

Below, I'm creating a new column called **year**, as we did before.

In [ ]:
median_close_time['year'] = median_close_time['Date_Opened'].dt.year

Oops! That didn't work because I forgot to reset the index. (Please don't copy these "mistakes" into your homework, lol.)

In [ ]:
median_close_time = median_close_time.reset_index()
median_close_time['year'] = median_close_time['Date_Opened'].dt.year

Renaming the columns:

In [ ]:
median_close_time.columns = ['date_opened', 'close_time', 'year']

In [ ]:
median_close_time

Subsetting the dataframe:

In [ ]:
median_close_time = median_close_time[['year', 'close_time']].copy()

In [ ]:
median_close_time

Let's make a chart!

In [ ]:
alt.Chart(median_close_time).mark_bar().encode(
    x='year:O',
    y='close_time',
).properties(
    title='Berkeley 311 calls: Median resolution time'
)

ARRGHHH! That didn't work. Let's look at the error: 
```
ValueError: Field "close_time" has type "timedelta64[ns]" which is not supported by Altair. Please convert to either a timestamp or a numerical value.
```
It sounds like I need to convert `timedelta` to a different unit. Let's try, er, nanoseconds.

In [ ]:
median_close_time['close_time_nanoseconds'] = median_close_time['close_time'].astype(int)
median_close_time

Let's try this again! I'm going to use a subset of the dataframe within the chart method argument because I don't want to create a whole new dataframe (Altair won't accept any dataframe at all with a dtype it can't support). Use your discretion for when you want to do something like this.

In [ ]:
alt.Chart(median_close_time[['year', 'close_time_nanoseconds']]).mark_bar().encode(
    x='year:O',
    y='close_time_nanoseconds',
).properties(
    title='Berkeley 311 calls: Median resolution time'
)

### Merge two dataframes

Now I'd like to merge `median_close_time` and `annual_cases`. Why? Mostly because I'd like to teach you how to merge dataframes. But you can get a neat summary table this way. Let's look at both dataframes again:

In [ ]:
annual_cases

In [ ]:
median_close_time

Let's look at the arguments in `pd.merge()` before we run it:

```python
pd.merge(
    df1,
    df2,
    how='outer', # other options: 'inner', 'left', 'right'
    on='Year',
    validate='1:1' # options: '1:m', 'm:m', 'm:1'
)
```
1. The first argument is the left-hand dataframe. The second argument is the right-hand dataframe. Why is it important that there's an order to dataframes? 

2. The `how` argument tells pandas how we'll merge the two dataframes. In this case, we'll use `outer`. But we could also use `left`, `right`, or `inner`. What does this mean? [Here are some visual examples of how joins work.](https://docs.google.com/spreadsheets/d/1SYukPLfuIkiqhIEPeXWXDBqClife8SoEgvHyBxw_ehs/edit) In this case, it doesn't matter which value we use for `how` because both dataframes have 10 rows with matching years. 

3. The `on` argument tells pandas which column key we're going to match on. In this case, we want the years to match up.

4. The `validate` argument is optional, but I recommend you learn how to use it. The value we used, `'1:1'` means that 1 row in the left-hand dataframe will match up to exactly 1 row in the right-hand dataframe. The option `1:m` means that 1 row in the left-hand dataframe could match up to **many** rows in the right-hand dataframe. (Any time you use `m`, you're telling pandas that there _might_ be multiple matches.)


In [ ]:
annual_summary = pd.merge(
    annual_cases,
    median_close_time,
    on='year',
    how='outer',
    validate='1:1'
)

In [ ]:
annual_summary

In [ ]:
annual_summary.to_csv('berkeley_311_annual_summary.csv', index=False)